In [78]:
import _init_paths

import numpy as np
import torch
import os
import pickle
from roi_data_layer.roidb import combined_roidb, rank_roidb_ratio, filter_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from torch.autograd import Variable


## load gt box

In [76]:
data_cache_path = 'data/cache'
imdb_classes =  ('__background__',  # always index 0
                          'person',
                          'people','cyclist'
                         )
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cpu'

In [71]:
class sampler(Sampler):
    def __init__(self, train_size, batch_size):
        self.num_data = train_size
        self.num_per_batch = int(train_size / batch_size)
        self.batch_size = batch_size
        self.range = torch.arange(0, batch_size).view(1, batch_size).long()
        self.leftover_flag = False
        if train_size % batch_size:
            self.leftover = torch.arange(self.num_per_batch * batch_size, train_size).long()
            self.leftover_flag = True

    def __iter__(self):
        rand_num = torch.randperm(self.num_per_batch).view(-1, 1) * self.batch_size
        self.rand_num = rand_num.expand(self.num_per_batch, self.batch_size) + self.range

        self.rand_num_view = self.rand_num.view(-1)

        if self.leftover_flag:
            self.rand_num_view = torch.cat((self.rand_num_view, self.leftover), 0)

        return iter(self.rand_num_view)

    def __len__(self):
        return self.num_data

In [60]:
def load_client_dataset(imdb_name):
    #dataloader_list = []
    #iter_epochs_list = []
    #for imdb_name in imdb_list:
    pkl_file = os.path.join(data_cache_path, imdb_name + '_gt_roidb.pkl')

    with open(pkl_file, 'rb') as f:
        roidb = pickle.load(f)

    roidb = filter_roidb(roidb)

    ratio_list, ratio_index = rank_roidb_ratio(roidb)

    train_size = len(roidb)
    print(train_size)
    iters_per_epoch = int(train_size / args.batch_size)
    print('iters_per_epoch: ' + str(iters_per_epoch))
    #iter_epochs_list.append(iters_per_epoch)
    sampler_batch = sampler(train_size, args.batch_size)

    dataset = roibatchLoader(roidb, ratio_list, ratio_index, args.batch_size, imdb_classes, training=True)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size,
                                             sampler=sampler_batch, num_workers=args.num_workers)
    #dataloader_list.append(dataloader)
    return dataloader,iters_per_epoch

In [67]:
class Arguments():
    def __init__(self):
        self.class_agnostic =False
        self.epochs = 3
        self.lr = 0.001
        self.lr_decay_step = 4
        self.dataset = 'KAIST'
        self.net = 'vgg16'
        self.batch_size = 24
        
        self.cuda = True
        #self.round = 10
        self.mGPUs=True
        self.optimizer ="sgd"
        self.k=3
        self.num_workers = 2
        #self.device = th.device("cpu")
        
args = Arguments()

In [74]:
imdb_name = 'KAIST_train'
dataloader,iters_per_epoch  = load_client_dataset(imdb_name)

before filtering, there are 43244 images...
after filtering, there are 43244 images...
43244
iters_per_epoch: 1801


In [79]:
data_iter = iter(dataloader)
data = next(data_iter)
im_data = torch.FloatTensor(1)
im_info = torch.FloatTensor(1)
num_boxes = torch.LongTensor(1)
gt_boxes = torch.FloatTensor(1)

# ship to cuda

im_data = im_data.to(device)
im_info = im_info.to(device)
num_boxes = num_boxes.to(device)
gt_boxes = gt_boxes.to(device)

# make variable
im_data = Variable(im_data)
im_info = Variable(im_info)
num_boxes = Variable(num_boxes)
gt_boxes = Variable(gt_boxes)

with torch.no_grad():
    im_data.resize_(data[0].size()).copy_(data[0])
    im_info.resize_(data[1].size()).copy_(data[1])
    gt_boxes.resize_(data[2].size()).copy_(data[2])
    num_boxes.resize_(data[3].size()).copy_(data[3])

In [21]:
from lib.model.rpn.generate_anchors import generate_anchors
anchor_scales= [4,8,16,32]

# Anchor ratios for RPN
ratios = [0.5,1,2]
_anchors = torch.from_numpy(generate_anchors(scales=np.array(anchor_scales), ratios=np.array(ratios))).float()
print(_anchors)
print(_anchors.size())


tensor([[ -38.,  -16.,   53.,   31.],
        [ -84.,  -40.,   99.,   55.],
        [-176.,  -88.,  191.,  103.],
        [-360., -184.,  375.,  199.],
        [ -24.,  -24.,   39.,   39.],
        [ -56.,  -56.,   71.,   71.],
        [-120., -120.,  135.,  135.],
        [-248., -248.,  263.,  263.],
        [ -14.,  -36.,   29.,   51.],
        [ -36.,  -80.,   51.,   95.],
        [ -80., -168.,   95.,  183.],
        [-168., -344.,  183.,  359.]])
torch.Size([12, 4])


In [12]:
feat_height, feat_width = 37,46
_feat_stride=16
shift_x = np.arange(0, feat_width) * _feat_stride
shift_y = np.arange(0, feat_height) * _feat_stride
shift_x, shift_y = np.meshgrid(shift_x, shift_y)
shifts = torch.from_numpy(np.vstack((shift_x.ravel(), shift_y.ravel(),
                          shift_x.ravel(), shift_y.ravel())).transpose())
shifts = shifts.contiguous().type(torch.cuda.FloatTensor).float()
                                     

In [20]:
print(shifts.size())
print(shifts)

torch.Size([1702, 4])
tensor([[  0.,   0.,   0.,   0.],
        [ 16.,   0.,  16.,   0.],
        [ 32.,   0.,  32.,   0.],
        ...,
        [688., 576., 688., 576.],
        [704., 576., 704., 576.],
        [720., 576., 720., 576.]], device='cuda:0')


In [24]:
K = shifts.size(0)
A = _anchors.size(0)
print(K)
print(A)

1702
12


In [47]:
(_anchors.view(1, A, 4) + shifts.view(K, 1, 4)).size()

torch.Size([1702, 12, 4])

In [30]:
_anchors = _anchors.type(torch.cuda.FloatTensor) # move to specific gpu.
all_anchors = _anchors.view(1, A, 4) + shifts.view(K, 1, 4)
all_anchors = all_anchors.view(K * A, 4)

total_anchors = int(K * A)
total_anchors

20424

In [51]:
_allowed_border = 0  
try:
    long        # Python 2
except NameError:
    long = int  # Python 3
keep = ((all_anchors[:, 0] >= -_allowed_border) &
                (all_anchors[:, 1] >= -_allowed_border) &
                (all_anchors[:, 2] < 750 + _allowed_border) &
                (all_anchors[:, 3] < 600 + _allowed_border))

torch.nonzero(keep)

tensor([[  588],
        [  600],
        [  612],
        ...,
        [19836],
        [19840],
        [19852]], device='cuda:0')

In [52]:
print(torch.nonzero(keep))

tensor([[  588],
        [  600],
        [  612],
        ...,
        [19836],
        [19840],
        [19852]], device='cuda:0')


In [39]:
keep[19852]

tensor(1, device='cuda:0', dtype=torch.uint8)

In [50]:
inds_inside = torch.nonzero(keep).view(-1)
inds_inside

tensor([  588,   600,   612,  ..., 19836, 19840, 19852], device='cuda:0')

In [82]:
from lib.model.rpn.bbox_transform import clip_boxes, bbox_overlaps_batch, bbox_transform_batch


In [83]:
inds_inside = torch.nonzero(keep).view(-1)

# keep only inside anchors
anchors = all_anchors[inds_inside, :]
batch_size=args.batch_size
# label: 1 is positive, 0 is negative, -1 is dont care
labels = gt_boxes.new(batch_size, inds_inside.size(0)).fill_(-1)  #[24, 9720]
bbox_inside_weights = gt_boxes.new(batch_size, inds_inside.size(0)).zero_()
bbox_outside_weights = gt_boxes.new(batch_size, inds_inside.size(0)).zero_()

overlaps = bbox_overlaps_batch(anchors, gt_boxes)

In [85]:
overlaps.size()

torch.Size([24, 9720, 20])

In [90]:
torch.nonzero(overlaps)


tensor([[   0, 1415,    4],
        [   0, 1423,    0],
        [   0, 1423,    2],
        ...,
        [  23, 7585,    0],
        [  23, 7590,    0],
        [  23, 7598,    0]], device='cuda:0')

In [94]:
overlaps[0][1415][4]

tensor(0.0034, device='cuda:0')

In [113]:
torch.nonzero(argmax_overlaps)

tensor([[   0, 1415],
        [   0, 1423],
        [   0, 1431],
        ...,
        [  20, 7816],
        [  20, 7824],
        [  20, 7832]], device='cuda:0')

In [115]:
argmax_overlaps[0][1415]  #記錄與anchor overlap最大的gt index

tensor(4, device='cuda:0')

In [119]:
max_overlaps[0][1415]#記錄與anchor overlap最大的ratio

tensor(0.0034, device='cuda:0')

In [103]:
max_overlaps, argmax_overlaps = torch.max(overlaps, 2)
gt_max_overlaps, _ = torch.max(overlaps, 1)
#max_overlaps.size()  #torch.Size([24, 9720])
#argmax_overlaps 24, 9720
gt_max_overlaps #[24,20] #gt找最大的overlap ratio


tensor([[0.7047, 0.5380, 0.3465, 0.5194, 0.3391, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.6389, 0.3169, 0.5897, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.3835, 0.4471, 0.3838, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.5221, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.5337, 0.5013, 0.5452, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.5946, 0.4905, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

In [107]:
#filter <0.3
RPN_NEGATIVE_OVERLAP=0.3
labels[max_overlaps < RPN_NEGATIVE_OVERLAP] = 0  #[24,9720] anchor數
torch.nonzero(labels).size()


torch.Size([2686, 2])

In [121]:
torch.nonzero(labels)

tensor([[   0, 3764],
        [   0, 4097],
        [   0, 4106],
        ...,
        [  23, 4460],
        [  23, 4799],
        [  23, 4808]], device='cuda:0')

In [141]:
temp_a = overlaps.eq(gt_max_overlaps.view(batch_size,1,-1).expand_as(overlaps))
print(torch.nonzero(temp_a))
print(temp_a[10][4411][3])

tensor([[   0, 4097,    4],
        [   0, 4106,    2],
        [   0, 4448,    4],
        [   0, 4457,    2],
        [   0, 4817,    0],
        [   0, 4896,    1],
        [   0, 4900,    3],
        [   0, 5247,    1],
        [   0, 5251,    3],
        [   0, 5598,    1],
        [   0, 5602,    3],
        [   0, 5953,    3],
        [   1, 3996,    1],
        [   1, 4347,    1],
        [   1, 6249,    2],
        [   1, 6277,    0],
        [   1, 6585,    2],
        [   1, 6613,    0],
        [   1, 6921,    2],
        [   1, 7257,    2],
        [   1, 7592,    2],
        [   2, 3966,    2],
        [   2, 4317,    2],
        [   2, 4484,    1],
        [   2, 4500,    0],
        [   2, 4668,    2],
        [   2, 4835,    1],
        [   2, 4851,    0],
        [   3, 4466,    0],
        [   4, 4508,    2],
        [   4, 4536,    1],
        [   4, 4874,    0],
        [   5, 2885,    1],
        [   5, 3184,    0],
        [   5, 3221,    1],
        [   6, 4448,

In [147]:
gt_max_overlaps[gt_max_overlaps==0] = 1e-5
keep = torch.sum(overlaps.eq(gt_max_overlaps.view(batch_size,1,-1).expand_as(overlaps)), 2)
#keep.size() #24,9720
torch.nonzero(keep)
keep[0][4097]

tensor(1, device='cuda:0')

In [150]:
if torch.sum(keep) > 0:
    labels[keep>0] = 1
    
RPN_POSITIVE_OVERLAP=0.7
# fg label: above threshold IOU
labels[max_overlaps >= RPN_POSITIVE_OVERLAP] = 1



num_fg = int(0.5 * 256)

sum_fg = torch.sum((labels == 1).int(), 1)
sum_bg = torch.sum((labels == 0).int(), 1)

In [153]:
labels.size()

torch.Size([24, 9720])

In [151]:
torch.sum((labels == 1).int(), 1)

tensor([12,  9,  7,  1,  3,  3,  2,  1,  5,  1, 15,  2, 25,  3,  3,  3,  1, 19,
        10,  4,  9,  1,  2,  2], device='cuda:0')